<a href="https://colab.research.google.com/github/johnnytorresm/stackoverflow/blob/main/fast_text2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ! pwd
import subprocess
p = subprocess.Popen(["pwd"], stdout=subprocess.PIPE)
output, err = p.communicate()
print("*** Running pwd command ***\n", output.decode('ascii'), err)

*** Running pwd command ***
 /content
 None


In [4]:
# Installing FastText

# # Cloning fastText from GitHub
# import os library
import os
os.system('git clone https://github.com/facebookresearch/fastText.git')
# change the current directory to specified directory
# os.chdir(r"./fastText")
print("Cloned")

Cloned


In [5]:

# p = subprocess.Popen(["cd", "fastText"], stdout=subprocess.PIPE)
# output, err = p.communicate()
# # print("*** Running cd command ***\n", output, err)
# os.chdir("fastText")
# print("Directory changed")

p = subprocess.Popen(["pwd"], stdout=subprocess.PIPE)
output, err = p.communicate()
print("*** Running pwd command ***\n", output.decode('ascii'), err)

p = subprocess.Popen(["ls", "-l"], stdout=subprocess.PIPE)
output, err = p.communicate()
print("*** Running ls -l command ***\n", output.decode('ascii'), err)

*** Running pwd command ***
 /content
 None
*** Running ls -l command ***
 total 8
drwxr-xr-x 13 root root 4096 Oct  3 16:08 fastText
drwxr-xr-x  1 root root 4096 Sep 30 17:12 sample_data
 None


In [7]:
os.chdir("fastText")
# p = subprocess.Popen(["cd", "fastText"], stdout=subprocess.PIPE)
# output, err = p.communicate()
print("*** Running cd command ***\n", output, err)
print("Directory changed")
os.system('make')

*** Running cd command ***
 b'total 8\ndrwxr-xr-x 13 root root 4096 Oct  3 16:08 fastText\ndrwxr-xr-x  1 root root 4096 Sep 30 17:12 sample_data\n' None
Directory changed


0

In [8]:
p = subprocess.Popen(["ls", "-l"], stdout=subprocess.PIPE)
output, err = p.communicate()
print("*** Running ls -l command ***\n", output.decode('ascii'), err)

*** Running ls -l command ***
 total 1288
drwxr-xr-x 2 root root   4096 Oct  3 16:08 alignment
-rw-r--r-- 1 root root  80592 Oct  3 16:10 args.o
-rw-r--r-- 1 root root 104976 Oct  3 16:10 autotune.o
-rwxr-xr-x 1 root root   1428 Oct  3 16:08 classification-example.sh
-rwxr-xr-x 1 root root   3154 Oct  3 16:08 classification-results.sh
-rw-r--r-- 1 root root   2214 Oct  3 16:08 CMakeLists.txt
-rw-r--r-- 1 root root   3356 Oct  3 16:08 CODE_OF_CONDUCT.md
-rw-r--r-- 1 root root   2061 Oct  3 16:08 CONTRIBUTING.md
drwxr-xr-x 2 root root   4096 Oct  3 16:08 crawl
-rw-r--r-- 1 root root  33536 Oct  3 16:10 densematrix.o
-rw-r--r-- 1 root root  96264 Oct  3 16:10 dictionary.o
drwxr-xr-x 2 root root   4096 Oct  3 16:08 docs
-rwxr-xr-x 1 root root   1287 Oct  3 16:08 download_model.py
-rw-r--r-- 1 root root   2102 Oct  3 16:08 eval.py
-rwxr-xr-x 1 root root 458120 Oct  3 16:10 fasttext
-rw-r--r-- 1 root root 205888 Oct  3 16:10 fasttext.o
-rw-r--r-- 1 root root    338 Oct  3 16:08 fasttext.pc.i

In [9]:
#-----------------------------
# Importing libraries
#-----------------------------
import numpy as np 
import pandas as pd
import re
import datetime
import glob
import string 
import io
from scipy import sparse
import csv
import codecs

import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

from datetime import timedelta
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

import fastText as ft

from collections import Counter

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split

import sklearn.metrics as metrics

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import ast
from collections import Counter

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

print('\nLibraries have been imported')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.

Libraries have been imported


In [5]:
! pwd

/content/fastText


In [11]:
from pathlib import Path
path = Path(os.getcwd())
parent = str(path.parent.absolute())
original_file = parent + "/corpus10k.csv"
p = subprocess.Popen(["cp", original_file, "."], stdout=subprocess.PIPE)
output, err = p.communicate()
print("*** copying files ***\n", output.decode('ascii'), err)
original_file = parent + "/other-stop-words.txt"
p = subprocess.Popen(["cp", original_file, "."], stdout=subprocess.PIPE)
output, err = p.communicate()
print("*** copying files ***\n", output.decode('ascii'), err)
p = subprocess.Popen(["ls", "-l"], stdout=subprocess.PIPE)
output, err = p.communicate()
print("*** Running ls -l command ***\n", output.decode('ascii'), err)

*** copying files ***
  None
*** copying files ***
  None
*** Running ls -l command ***
 total 13396
drwxr-xr-x 2 root root     4096 Oct  3 16:08 alignment
-rw-r--r-- 1 root root    80592 Oct  3 16:10 args.o
-rw-r--r-- 1 root root   104976 Oct  3 16:10 autotune.o
-rwxr-xr-x 1 root root     1428 Oct  3 16:08 classification-example.sh
-rwxr-xr-x 1 root root     3154 Oct  3 16:08 classification-results.sh
-rw-r--r-- 1 root root     2214 Oct  3 16:08 CMakeLists.txt
-rw-r--r-- 1 root root     3356 Oct  3 16:08 CODE_OF_CONDUCT.md
-rw-r--r-- 1 root root     2061 Oct  3 16:08 CONTRIBUTING.md
-rw-r--r-- 1 root root 12393293 Oct  3 16:14 corpus10k.csv
drwxr-xr-x 2 root root     4096 Oct  3 16:08 crawl
-rw-r--r-- 1 root root    33536 Oct  3 16:10 densematrix.o
-rw-r--r-- 1 root root    96264 Oct  3 16:10 dictionary.o
drwxr-xr-x 2 root root     4096 Oct  3 16:08 docs
-rwxr-xr-x 1 root root     1287 Oct  3 16:08 download_model.py
-rw-r--r-- 1 root root     2102 Oct  3 16:08 eval.py
-rwxr-xr-x 1 roo

In [12]:
! ls -l /content/*.txt

-rw-r--r-- 1 root root 1090 Oct  3 16:13 /content/other-stop-words.txt


In [13]:
! ls -l /content/*.csv

-rw-r--r-- 1 root root 12393293 Oct  3 16:13 /content/corpus10k.csv


In [15]:
#--------------------
# Global Variables
#--------------------
sw  = set(stopwords.words())

# Exhaustive list of English prepositions to be added to stop words
preps = pd.read_csv('/content/other-stop-words.txt', header=None, names=['other-stop-words'])
preps_set = set(preps['other-stop-words'].tolist())
sw |= preps_set

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

print('Global Variables have been defined')


Global Variables have been defined


In [16]:
#-----------------------------------------
# Function definitions
#-----------------------------------------

#----------------------------------------------------
# clean_txt: Cleans a string, and then lemmatize it.
#----------------------------------------------------
def clean_txt(txt): 
    # txt: string to clean
    
    global sw, lemmatizer                # sw = English stop words; lemmatizer from WordNet

    # 1. Lowercase 
    txt = txt.lower()

    # 2. Strip HTML code
    txt = re.sub('<[^<]+?>', ' ', txt)

    # 3. Strip punctuation signs
    txt = re.sub('[^\w\s]', ' ', txt)
    txt = re.sub('\*', ' ', txt)

    # 4. remove stop words
    txt_set = set(nltk.word_tokenize(txt))
    txt_lst = [w for w in txt_set if (not w in sw)]
    txt_set = set(txt_lst)
    txt_lst = list(txt_set)
    
    # 5. Lemmatisation
    txt_lst = [lemmatizer.lemmatize(w, pos='v') for w in txt_lst]
    txt = ' '.join([lemmatizer.lemmatize(w, pos='n') for w in txt_lst])
    
    return txt

#-----------------------------------------
# display_topics after LDA
#-----------------------------------------
def display_topics(model, feature_names, no_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        topic_list = " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words-1:-1]])
        print(topic_list)
        topics.append(topic_list)
    return topics
    
#-----------------------------------------
# Print a classifier's scores
#-----------------------------------------
def print_score(y_true, y_pred, clf):
    print("Classifier: ", clf.__class__.__name__)
    print("Precision score : {}".format(precision_score(y_true, y_pred, average='weighted')))
    print("Recall score : {}".format(recall_score(y_true, y_pred, average='weighted')))
    print("F1 score : {}".format(f1_score(y_true, y_pred, average='weighted')))

#-----------------------------------------
# Print fasttext's scores
#-----------------------------------------
def ft_results(N, p, r):
    print("N\t" + str(N))
    f1 = 2 * ( p * r ) / ( p + r)   # calculate f1 score
    print("P@{}\t{:.4f}".format(1, p))
    print("R@{}\t{:.4f}".format(1, r))
    print("F1@{}\t{:.4f}".format(1, f1))
    
#-----------------------------------------
# Receives a list of tags and returns
# a list of Labels in fasttext format
#-----------------------------------------
def labels(Tags):
    y_labels = ' '
    for t in Tags:
        y_labels += '__label__'+ t + ' '
    
    return y_labels

#-----------------------------------------
# most frequent tags in the corpus
#-----------------------------------------
def most_used_tags(tags, top_tags):
    final_tags = []
    for tag in tags:
        if tag in top_tags['Tags'].values:
            final_tags.append(tag)
    return final_tags

#-----------------------------------------
# Select file from a directory
#-----------------------------------------
def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True) # List of selected files will be set button's file attribute.
    print(b.files) # Print the list of files selected.
    
#-----------------------------------------
# Remove "__label__" from FastText Labels
#-----------------------------------------
def remove_label(l,prefix)-> str:
    if l.startswith(prefix):
        return l[len(prefix):]
    else:
        return l[:]

#---------------------------------------------
# Sorts a dictionary by value. Returns a list.
#---------------------------------------------
def sort_tags(x):
    if len(x)>0:
        s = sorted(x.items(), key = lambda y:(y[1], y[0]), reverse = True)
    else:
        s = []
    return s

#---------------------------------------------
# Top 5 values 
#---------------------------------------------
def top_tags(x):
    s = []
    i = 0
    if len(x)>0:
        while ((i<len(x)) and (i<5)):
            s = s + [x[i][0]]
            i += 1
    else:
        s = []
    
    return s

print('Functions have been defined')


Functions have been defined


In [17]:
#--------------------------------
# Reading data
#--------------------------------

file2open = "/content/corpus10k.csv"

print('Reading...')
posts = pd.read_csv(file2open, usecols=['Id', 'Tags', 'Text'])

# Number of records read
display(posts.shape)
display(posts.head())
        
# counting nulls per column
display(posts.isnull().sum())


Reading...


(11146, 3)

,Id,Tags,Text
0,4,"['c#', 'floating-point', 'type-conversion', 'd...",change error vb use work 5000 track try contro...
1,6,"['html', 'css', 'internet-explorer-7']",collapse miss explorer 0 children specificatio...
2,9,"['c#', '.net', 'datetime']",someone datetime represent base age calculate ...
3,11,"['c#', 'datetime', 'time', 'datediff', 'relati...",display datetime days 2 time month 3 specific ...
4,13,"['html', 'browser', 'timezone', 'user-agent', ...",timezone http part agent determine user string...


Id      0
Tags    0
Text    0
dtype: int64

In [18]:
#----------------------------------------------------------------
# Checking if Tags have been read as a single string or a series
#-----------------------------------------------------------------
if isinstance(posts['Tags'].iloc[0],str): # verifies if 'Tags' is of type "string"
    display(posts['Tags'].iloc[0])
    posts['Tags'] = posts['Tags'].apply(lambda tag: ast.literal_eval(tag))
    display(posts['Tags'].iloc[0])

print('Ready')

"['c#', 'floating-point', 'type-conversion', 'double', 'decimal']"

['c#', 'floating-point', 'type-conversion', 'double', 'decimal']

Ready


In [19]:


#------------------------------------------------------
# Flatten the "Tags" columns into a Series and then 
# count the number of occurrences per item, putting
# the results into a dataframe
#------------------------------------------------------
tag_series = pd.Series([item for sublist in posts['Tags'] for item in sublist])
tag_df = tag_series.groupby(tag_series).size().rename_axis('Tags').reset_index(name='Nº of occurrences')
tag_df = tag_df.sort_values(by=['Nº of occurrences'], ascending=False)
tag_df.head(20)

# 2
# Nº of Tags that appear more than 100 times across all messages

series = tag_df['Nº of occurrences'].apply(lambda x: True if x > 10 else False)
counting = len(series[series == True].index)

print('Nº of Tags that appear more than 10 times across all messages:', counting)


Nº of Tags that appear more than 10 times across all messages: 454


In [20]:
#-----------------------------------------
# Top 100 tags
#-----------------------------------------
top_tags = tag_df[['Tags', 'Nº of occurrences']].head(100)
display(top_tags)

#--------------------------
# y = list of Tags list
#--------------------------
posts['Tags'] = posts['Tags'].apply(lambda tags: most_used_tags(tags, top_tags))
posts = posts.loc[posts['Tags'].str.len() > 0]
y = posts['Tags']
print('Ready - Nº of chosen tags', len(y))

#--------------------
# Display results
#--------------------
display(posts.head()), display(y)


,Tags,Nº of occurrences
407,c#,1370
2,.net,1237
1777,java,807
195,asp.net,693
410,c++,527
...,...,...
3760,vba,50
839,date,49
1639,image,49
811,database-design,49


Ready - Nº of chosen tags 9591


,Id,Tags,Text
0,4,[c#],change error vb use work 5000 track try contro...
1,6,"[html, css]",collapse miss explorer 0 children specificatio...
2,9,"[c#, .net]",someone datetime represent base age calculate ...
3,11,[c#],display datetime days 2 time month 3 specific ...
4,13,"[html, browser]",timezone http part agent determine user string...


0                               [c#]
1                        [html, css]
2                         [c#, .net]
3                               [c#]
4                    [html, browser]
                    ...             
11139          [c++, multithreading]
11140                       [vb.net]
11141    [.net, asp.net, javascript]
11142              [javascript, oop]
11145                     [security]
Name: Tags, Length: 9591, dtype: object

(None, None)

In [21]:
#------------------------------------
# Apply MultiLabelBinarizer to Tags
#------------------------------------
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(posts['Tags'])
y[0:10,0:10]


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [22]:
#------------
# TD-IDF 
#------------
tfidf = TfidfVectorizer(analyzer="word", max_features=1000, ngram_range=(1,1))
X = tfidf.fit_transform(posts['Text'])

print('Ready')


Ready


In [23]:
#------------------------------
# Split into train and test
#------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape,  X_test.shape,  y_train.shape, y_test.shape
# 

((7672, 1000), (1919, 1000), (7672, 100), (1919, 100))

In [24]:
#-------------------------------------------------------
# Preparation of tags for fasttext 
# The string "__label__" is attached to each tag to create a new column named 'Labels'.
# Columns 'Labels' and 'Text' are then written to an external txt file for fasttext analysis (in OS mode)
#-------------------------------------------------------
train, test = train_test_split(posts, test_size=0.2)

train['Labels'] = train['Tags'].apply(lambda x: labels(x))
test['Labels'] = test['Tags'].apply(lambda x: labels(x))

train = train.dropna()
test = test.dropna()

display(train[['Tags', 'Labels', 'Text']].head(10))
display(test[['Tags', 'Labels', 'Text']].head(10))


,Tags,Labels,Text
60,"[java, generics]",__label__java __label__generics,cast check information getmylist gt object cas...
11078,[ruby],__label__ruby,error saw modify say classtwo work delete_all ...
7529,"[c#, .net]",__label__c# __label__.net,responsestream ipaddress ip respond great send...
1700,[algorithm],__label__algorithm,card display speed run render fast game logic ...
6699,"[database, database-design]",__label__database __label__database-design,insist argue efficiency even table outweigh fl...
3573,"[c#, javascript, web-services]",__label__c# __label__javascript __label__web-...,change use 1 old strange long deprecate htc co...
3335,"[sql-server, ruby-on-rails, ruby, sharepoint]",__label__sql-server __label__ruby-on-rails __...,various display webui even rail sharepoint dep...
10661,"[c#, .net]",__label__c# __label__.net,always reason work seem time method applicatio...
7916,"[ruby, exception]",__label__ruby __label__exception,get http seem look nameerror aws implicit pres...
7502,[php],__label__php,create usually helper structure build differen...


,Tags,Labels,Text
6097,"[asp.net, security, email]",__label__asp.net __label__security __label__e...,concern http 10 aspx email php use archive mai...
6266,[svn],__label__svn,date root otherwise however scan tag 721 2008 ...
9527,[sql-server],__label__sql-server,change register sql register port however see ...
9762,[asp.net],__label__asp.net,error suggestion get aspx file mappings 0 mess...
5146,[c#],__label__c#,operator get automatically hold throw explain ...
3264,"[php, iis]",__label__php __label__iis,urls spirit http windows p work team use php h...
613,"[windows, visual-studio]",__label__windows __label__visual-studio,graphical view file item button let tfs items ...
10005,"[c#, .net]",__label__c# __label__.net,maintain information read assemblies tedious f...
4407,[python],__label__python,split variable say dictionary sql function 0 f...
1771,"[sql, debugging]",__label__sql __label__debugging,debug tool sql recommend look debugger standal...


In [25]:
#-------------------------------------------------------
# It writes Labels and Text in FastText format to train.txt and test.txt files
# Those files will be the input for FastText routines
#-------------------------------------------------------

train_file = os.path.splitext(file2open)[0] + '.train'
test_file  = os.path.splitext(file2open)[0] + '.test'

train.to_csv(train_file, 
             header=None, 
             index=None, 
             mode='a', 
             encoding='utf-8', 
             columns=['Labels', 'Text'],
             sep=' ',
             escapechar=' ',
             quoting=csv.QUOTE_NONE,
             quotechar=' '
            )

print('Ready ', train_file)

test.to_csv(test_file,
            header=None, 
            index=None, 
            mode='a', 
            encoding='utf-8', 
            columns=['Labels', 'Text'],
            sep=' ',
            escapechar=' ',
            quoting=csv.QUOTE_NONE,
            quotechar=' '
           )

print('Ready ', test_file)

Ready  /content/corpus10k.train
Ready  /content/corpus10k.test


In [26]:
#-------------------------------------------------------
# Training the FastText model
#-------------------------------------------------------

print('FastText training Begin...', datetime.datetime.now())

# Command for training 

filename = os.path.splitext(file2open)[0]

input_file = filename + '.train'

output_file = filename

result_file = filename + ".result"

batcmd = './fasttext supervised -input '+ input_file + ' -output ' + \
                                          output_file + ' -dim 10 -lr 1 -wordNgrams 1 -minCount 1 -bucket 10000000 -epoch 25' + \
                                          '> ' + result_file

result = subprocess.check_output(batcmd, shell=True)

print('FastText training End...', datetime.datetime.now())

print('Ready')


FastText training Begin... 2021-10-03 16:17:05.913977
FastText training End... 2021-10-03 16:17:08.419809
Ready


In [27]:
! ls -l /content/*.bin

-rw-r--r-- 1 root root 1618139 Oct  3 16:17 /content/corpus10k.bin


In [28]:
#-------------------------------------------------------
# Testing the fast text model
#-------------------------------------------------------

# Command for testing

input_file = filename + '.bin'

test_file  = filename + '.test'

# batcmd = 'fasttext test '+ input_file + ' ' + output_file + ' 2>$null'

batcmd = './fasttext test '+ input_file + ' ' + test_file 

print('fast text Begin...', datetime.datetime.now())

result = subprocess.check_output(batcmd, shell=True)

print('fast text End...', datetime.datetime.now())

print(result.decode('utf-8'))


fast text Begin... 2021-10-03 16:18:13.767609
fast text End... 2021-10-03 16:18:13.825881
N	1919
P@1	0.521
R@1	0.297



In [30]:
#-------------------------------------------------------
# Predicting labels
#-------------------------------------------------------

# File where question is written
question_file = open("question.txt", "w")

# File that contains the model
input_file = filename + '.bin'

# User is asked
question = input("Please, enter your question:\n")

# Question is cleaned and lemmatized
question2 = clean_txt(question)

# Question is written to a file
n = question_file.write(question2)

# Question file is closed
question_file.close()

# FastText string command is assembled
batcmd = './fasttext predict '+ input_file + ' question.txt 5 1>labels.csv'

# FastText commnd is executed. Output written to "labels.csv"
result = subprocess.check_output(batcmd, shell=True)

# labels are read and split into a list
csv_reader = csv.reader(codecs.open('labels.csv', 'rU', 'utf-8'))

lists_from_csv = []

for row in csv_reader:
    lists_from_csv.append(row)

list_from_csv = list(tuple(remove_label(l,'__label__') for l in lists_from_csv[0][0].split()))

print(list_from_csv)

Please, enter your question:
I want to write a program in Python that sends a message from a smartphone to the screens of several computers running different operating systems like Windows, Unix, MacOs, etc. across a TCP/IP network.
['networking', 'macos', 'unix', 'winapi', 'bash']
